![JohnSnowLabs](https://nlp.johnsnowlabs.com/assets/images/logo.png)

[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/JohnSnowLabs/spark-nlp-workshop/blob/master/tutorials/streamlit_notebooks/healthcare/NER_BIOMARKER.ipynb)

## **Detect Radiology Entities**

To run this yourself, you will need to upload your license keys to the notebook. Just Run The Cell Below in order to do that. Also You can open the file explorer on the left side of the screen and upload license_keys.json to the folder that opens. Otherwise, you can look at the example outputs at the bottom of the notebook.

## **Colab Setup**

In [ ]:
# Install the johnsnowlabs library to access Spark-OCR and Spark-NLP for Healthcare, Finance, and Legal.
! pip install johnsnowlabs

In [ ]:
from google.colab import files
print("Please Upload your John Snow Labs License using the button below")
license_keys = files.upload()

In [ ]:
from johnsnowlabs import *

# After uploading your license run this to install all licensed Python Wheels and pre-download Jars the Spark Session JVM
# Make sure to restart your notebook afterwards for changes to take effect

jsl.install()

## Start Session

In [ ]:
from johnsnowlabs import *
# Automatically load license data and start a session with all jars user has access to
spark = jsl.start()

In [ ]:
spark

# **🔎Define Spark NLP pipeline**

In [ ]:
documentAssembler = nlp.DocumentAssembler()\
        .setInputCol("text")\
        .setOutputCol("document")
        
sentenceDetector = nlp.SentenceDetectorDLModel.pretrained() \
      .setInputCols(["document"]) \
      .setOutputCol("sentence") 
 
tokenizer = nlp.Tokenizer()\
        .setInputCols(["sentence"])\
        .setOutputCol("token")

word_embeddings = nlp.WordEmbeddingsModel.pretrained("embeddings_clinical","en","clinical/models")\
        .setInputCols(["sentence","token"])\
        .setOutputCol("embeddings")

clinical_ner = medical.NerModel.pretrained("ner_biomarker", "en", "clinical/models") \
  .setInputCols(["sentence", "token", "embeddings"]) \
  .setOutputCol("ner")

ner_converter = nlp.NerConverter()\
        .setInputCols(["sentence","token","ner"])\
        .setOutputCol("ner_chunk")

nlpPipeline = Pipeline(stages=[documentAssembler,
                               sentenceDetector,
                               tokenizer,
                               word_embeddings,
                               clinical_ner,
                               ner_converter])

sentence_detector_dl download started this may take some time.
Approximate size to download 354.6 KB
[OK!]
embeddings_clinical download started this may take some time.
Approximate size to download 1.6 GB
[OK!]
ner_biomarker download started this may take some time.
[OK!]


## **Create example inputs**

In [ ]:
sample_text = ["""All the tumor tissues contained small cell carcinoma components. 4 cases coexisted with other histologic types of bladder cancers, and 2 out of the 9 cases had three different cell components. All the patients had muscle invasion, and 4 cases showed lymph nodes metastasis, 3 cases showed invasion of neighboring structures (seminal vesicle or uterus), and 1 case was highly suspected of liver metastasis. Immunohistochemistry results showed that PCK, Syn, NSE, and CD56 were all positive, but LCA was negative."""]

## **Use the pipeline to create outputs**

In [ ]:
from pyspark.sql.types import StringType, IntegerType

df = spark.createDataFrame(sample_text,StringType()).toDF('text')
result = nlpPipeline.fit(df).transform(df)

result.select(F.explode(F.arrays_zip(result.ner_chunk.result, 
                                     result.ner_chunk.metadata)).alias("cols"))\
      .select(F.expr("cols['0']").alias("chunk"),
              F.expr("cols['1']['entity']").alias("ner_label")).show(30, truncate=False)

+--------------------+---------------------+
|chunk               |ner_label            |
+--------------------+---------------------+
|tumor               |Tumor_Finding        |
|small cell          |CancerModifier       |
|carcinoma           |CancerDx             |
|bladder cancers     |CancerDx             |
|metastasis          |Metastasis           |
|liver metastasis    |Metastasis           |
|Immunohistochemistry|Test                 |
|PCK                 |Biomarker            |
|Syn                 |Biomarker            |
|NSE                 |Biomarker            |
|CD56                |Biomarker            |
|positive            |Biomarker_Measurement|
|LCA                 |Biomarker            |
|negative            |Biomarker_Measurement|
+--------------------+---------------------+



## **Visualize results**

In [ ]:
from sparknlp_display import NerVisualizer

NerVisualizer().display(
    result = result.collect()[0],
    label_col = 'ner_chunk',
    document_col = 'document'
)